In [1]:
# We'll the steps as before, 1) divide the corpus into sentences 2) tokenize the sentences 3) get_vocab to make the one_hot encoding
# add starts and ends

In [2]:
import wikipedia
import math
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F


In [3]:

topics = [
    "Artificial Intelligence",
    "Climate Change",
    "Quantum Computing",
    "World War II",
    "Ancient Egypt",
    "Space Exploration",
    "Global Health",
    "Economics",
    "Philosophy of Science",
    "Modern Art",
    "Genetics",
    "Renewable Energy",
    "Cybersecurity",
    "Cryptocurrency",
    "Social Media",
    "Cultural Anthropology",
    "Astrophysics",
    "Human Rights",
    "Machine Learning",
    "History of Technology",
    "Biotechnology"
]

In [4]:
def divide_corpus(corpus):
    sentences=corpus.split('.')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    return sentences
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        tokenized = sentence.split(' ')
        final_tokenized=[]
        for word in tokenized:
            if len(word)>0:
                if (word[-1]>='a' and word[-1]<='z') or word[-1]=='>':
                    final_tokenized.append(word)
                else:
                    final_tokenized.append(word[:-1])
        tokenized_sentences.append(final_tokenized)
        
    
    return tokenized_sentences

def get_vocabulary(data,min_freq=1):
    word_counts={}  #keys:word and value:count
    vocab={}
    for sentence in data:
        for word in sentence:
            if word in word_counts.keys():
                word_counts[word]+=1
            else:
                word_counts[word]=1
    i=0
    for key,value in word_counts.items():
        if value>=min_freq:
            vocab[key]=i
            i+=1
    return vocab
def add_starts_ends(data,n=1):
    final_data=[]
    for sentence in data:
        final_data.append(['<s>'] * n + sentence + ['<e>'])
    return final_data
def suggest_word(sentence,lm):
    max_prob=-10000
    suggested_word=''
    sentence_log_prob=sum([np.log(lm[(sentence[i],sentence[i+1])]) for i in range(len(sentence)-1) if (sentence[i],sentence[i+1]) in lm.keys()])
    for key,value in lm.items():
        if key[0]==sentence[len(sentence)-1]:
            if np.log(value)+sentence_log_prob>max_prob:
                max_prob=np.log(value)+sentence_log_prob
                suggested_word=key[1]
    return suggested_word

def one_hot_encode(word,vocab):
    word_encoded=np.zeros(len(vocab))
    word_encoded[vocab[word]]=1
    return(word_encoded)
def get_input(sentence,i,vocab):
    sentence_encoded=np.zeros(len(vocab))
    for j in range(0,i):
        sentence_encoded+=one_hot_encode(sentence[j],vocab)
    return(sentence_encoded/(j+1))

def prepare_train_data(data,vocab):
    y=[]
    x=[]
    for sentence in data:
        for i in range(1,len(sentence)):
            y.append(one_hot_encode(sentence[i],vocab))
            x.append(get_input(sentence,i,vocab))
    return np.array(x),np.array(y)

In [5]:
corpus=''
for topic in topics:
    try:
        page = wikipedia.page(topic)
        corpus+=page.content
        print(topic)
    except:
        print('skipped:',topic)
        continue

Artificial Intelligence
Climate Change
Quantum Computing
World War II
Ancient Egypt
skipped: Space Exploration
Global Health
Economics
Philosophy of Science
Modern Art


/home/mahdi/anaconda3/envs/myenv/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/mahdi/anaconda3/envs/myenv/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


skipped: Genetics
Renewable Energy
Cybersecurity
Cryptocurrency
Social Media
Cultural Anthropology
Astrophysics
Human Rights
skipped: Machine Learning
History of Technology
Biotechnology


In [6]:
data=divide_corpus(corpus)
data=tokenize_sentences(data)
data=add_starts_ends(data)
vocab=get_vocabulary(data,1)
print(len(vocab))

18385


In [8]:
device="cuda"
class LLM(nn.Module):
    def __init__(self,in_features):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.ReLU(),
            nn.Linear(64, in_features),
            nn.Softmax()
        )
    def forward(self,x):
        return self.net(x)
in_features=len(vocab)
model=LLM(in_features)

In [18]:
EPOCHS=200
BATCH_SIZE=32
best_accuracy=0
crit=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=1e-2,weight_decay=5e-4)
for epoch in range(EPOCHS):
    j=random.randint(0,len(vocab)-BATCH_SIZE)
    x,y=prepare_train_data(data[0:BATCH_SIZE],vocab)
    x=torch.from_numpy(x).float()
    y=torch.from_numpy(y).float()
    #forward
    y_hat=model(x)
    loss=crit(y,y_hat)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, predicted = torch.max(y_hat, 1)
    y_indices = torch.argmax(y, dim=1)
    correct = (predicted == y_indices).sum().item()
    total = BATCH_SIZE
    
    accuracy = correct / total
    if accuracy>=best_accuracy:
        best_accuracy=accuracy
    print(epoch)
    print('accuracy: ',accuracy)
    print('best_accuracy: ',best_accuracy)

0
accuracy:  0.96875
best_accuracy:  0.96875
1
accuracy:  0.0
best_accuracy:  0.96875
2
accuracy:  0.53125
best_accuracy:  0.96875
3
accuracy:  0.53125
best_accuracy:  0.96875
4
accuracy:  1.0
best_accuracy:  1.0
5
accuracy:  1.0
best_accuracy:  1.0
6
accuracy:  0.96875
best_accuracy:  1.0
7
accuracy:  0.96875
best_accuracy:  1.0
8
accuracy:  0.96875
best_accuracy:  1.0
9
accuracy:  0.96875
best_accuracy:  1.0
10
accuracy:  0.96875
best_accuracy:  1.0
11
accuracy:  0.96875
best_accuracy:  1.0
12
accuracy:  1.0
best_accuracy:  1.0
13
accuracy:  1.0
best_accuracy:  1.0
14
accuracy:  1.0
best_accuracy:  1.0
15
accuracy:  1.0
best_accuracy:  1.0
16
accuracy:  1.0
best_accuracy:  1.0
17
accuracy:  0.96875
best_accuracy:  1.0
18
accuracy:  0.96875
best_accuracy:  1.0
19
accuracy:  0.96875
best_accuracy:  1.0
20
accuracy:  0.96875
best_accuracy:  1.0
21
accuracy:  0.96875
best_accuracy:  1.0
22
accuracy:  0.96875
best_accuracy:  1.0
23
accuracy:  1.0
best_accuracy:  1.0
24
accuracy:  1.0
best

KeyboardInterrupt: 